In [1]:
import single_head
import multi_head
import copy
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage.morphology import binary_fill_holes
import sys
import pickle
np.set_printoptions(threshold=sys.maxsize)

%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [33]:
# load heads
all_heads=[single_head.SingleHead.load_from_pickle(2,i) for i in range (1,16)]

In [34]:
# make head from first to last
def make_mhead(all_heads, first, last):
    all_heads[first].reset_positions()
    all_heads[first].reset_colors()
    all_heads[first+1].reset_positions()
    all_heads[first+1].reset_colors()   
    heads = multi_head.MultiHead.joined_heads(all_heads[first], all_heads[first+1])
    heads.links.append(multi_head.Link(left=all_heads[first+1].frame_id,right=all_heads[first].frame_id))
    for i in range(first+2,last+1):
        heads.links.append(multi_head.Link(left=all_heads[i].frame_id,right=all_heads[i-1].frame_id))
        all_heads[i].reset_positions()
        all_heads[i].reset_colors()
        heads.append(all_heads[i])        
        if i==last:
             heads.links.append(multi_head.Link(left=all_heads[first].frame_id,right=all_heads[i].frame_id))
    print ("completed")
    return heads
mhead = make_mhead(all_heads, 0,14)
mhead.calc_keypoints()

completed


In [35]:
def calc_connections(mhead):
    for link in mhead.links:
        link.reset()
        mhead.ransac_from_link(link)        
calc_connections(mhead)

In [36]:
only_first_n=15
mhead.reset_all_head_positions()
link_index,err=mhead.get_next_unpositioned_link()
positioned_head_count=0
joined_heads=set() 
while(not link_index is None) and (positioned_head_count<only_first_n or only_first_n==-1):
    mhead.links[link_index].print_short()
    joined_heads.add(mhead.links[link_index].left)
    joined_heads.add(mhead.links[link_index].right)
    foo= mhead.transform_from_link(mhead.links[link_index])
    link_index,err=mhead.get_next_unpositioned_link()
    positioned_head_count=max(positioned_head_count+1,2)
joined_heads=sorted(joined_heads)
print(joined_heads)
no_heads= len(joined_heads)
joined_heads=[str(h) for h in joined_heads]
name= f"Seq_{mhead.heads[0].sequence_id}_frames_{'_'.join(joined_heads)}"
mhead.create_spheres(min(2/max(no_heads,0.2),1),name=name)

any_head_positioned False
15-14, matches=1060, Count=927, Err=0.0012
any_head_positioned True
1-15, matches=1017, Count=851, Err=0.0015
any_head_positioned True
2-1, matches=967, Count=820, Err=0.0015
any_head_positioned True
3-2, matches=812, Count=592, Err=0.0024
any_head_positioned True
4-3, matches=702, Count=494, Err=0.0027
any_head_positioned True
5-4, matches=652, Count=449, Err=0.0028
any_head_positioned True
6-5, matches=606, Count=462, Err=0.0023
any_head_positioned True
14-13, matches=741, Count=473, Err=0.0031
any_head_positioned True
7-6, matches=308, Count=172, Err=0.0037
any_head_positioned True
8-7, matches=168, Count=109, Err=0.0025
any_head_positioned True
9-8, matches=134, Count=86, Err=0.0023
any_head_positioned True
10-9, matches=161, Count=64, Err=0.0037
any_head_positioned True
11-10, matches=165, Count=85, Err=0.0023
any_head_positioned True
13-12, matches=211, Count=24, Err=0.0159
any_head_positioned True
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [37]:
mhead.create_mesh('mesh')


completed


In [85]:
for i, link in enumerate(mhead.links):
    print (i)
    link.print_short()

0
2-1, Count=78, Err=0.0029
1
3-2, Count=86, Err=0.0037
2
4-3, Count=53, Err=0.0053
3
5-4, Count=80, Err=0.0028
4
6-5, Count=67, Err=0.0051
5
7-6, Count=52, Err=0.0055
6
8-7, Count=39, Err=0.0030
7
9-8, Count=12, Err=0.0130
8
10-9, Count=5, Err=nan
9
11-10, Count=19, Err=0.0028
10
12-11, Count=27, Err=0.0076
11
13-12, Count=39, Err=0.0030
12
14-13, Count=34, Err=0.0068
13
15-14, Count=42, Err=0.0076
14
1-15, Count=92, Err=0.0028


In [100]:
mhead.reset_all_head_positions()
mhead.transform_from_link(mhead.links[13])
mhead.create_spheres()

In [32]:
mhead.create_mesh("mesh_head")


completed


In [ ]:
# bad 11 12
# 12-11, Count=53, Err=0.0077

second

In [ ]:
#using edge diameter 20, 2 nearest neighbors, 0.9 threshold
#using edge diameter 20, 1 nearest neighbors, 0.4 threshold

In [31]:
mhead.reset_all_head_positions()
maxhead = 15
for idx in range(maxhead):
    mhead.transform_from_link(mhead.links[idx])
    mhead.links[idx].print_short()
center, distances=mhead.left_eye_deviation()
np.mean(distances)

mhead.create_spheres(0.5,"seq_2_mergeSet_{}".format(maxhead))

2-1, matches=967, Count=820, Err=0.0015
3-2, matches=812, Count=591, Err=0.0024
4-3, matches=702, Count=494, Err=0.0027
5-4, matches=652, Count=445, Err=0.0028
6-5, matches=606, Count=461, Err=0.0023
7-6, matches=308, Count=177, Err=0.0038
8-7, matches=168, Count=109, Err=0.0025
9-8, matches=134, Count=85, Err=0.0024
10-9, matches=161, Count=64, Err=0.0036
11-10, matches=165, Count=85, Err=0.0023
12-11, matches=85, Count=14, Err=0.0160
13-12, matches=211, Count=28, Err=0.0145
14-13, matches=741, Count=474, Err=0.0031
15-14, matches=1060, Count=927, Err=0.0012
1-15, matches=1017, Count=851, Err=0.0015
0 153974
coordinate of the left eye: [ 0.00277377 -0.09670738 -0.06905971] in frame 0
1 150143
coordinate of the left eye: [ 0.03848441 -0.14546269 -0.00836436] in frame 1
2 150124
coordinate of the left eye: [ 0.03808846 -0.14535251 -0.00839116] in frame 2
13 147576
coordinate of the left eye: [ 0.00752822 -0.09694682 -0.0707703 ] in frame 13
14 150775
coordinate of the left eye: [ 0.0041